In [7]:
import json
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer

nltk.download('wordnet_ic')
brown_ic = wordnet_ic.ic('ic-brown.dat')
from itertools import product


[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/viviancai/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [8]:
true_set = set()
false_set = set()

In [9]:


def is_synonym(word1, word2, word_syn_set):
#     print(word1, word2)
    key1 = word1+" "+word2
    key2 = word2+" "+word1
    if key1 in true_set or key2 in true_set:
#         print("used true set")
        return True

    if key1 in false_set or key2 in false_set:
#         print("used false set")
        return False
    
    if word2 in word_syn_set:
        return True
    if len(word1)>=3 and (word2.startswith(word1) or word2.endswith(word1)):
        return True
    if len(word2)>=3 and (word1.startswith(word2) or word1.endswith(word2)):
        return True
    
    pos_tags = [wn.NOUN, wn.ADJ]
    for pos_tag in pos_tags:
        syns1 = wn.synsets(word1, pos=pos_tag)
        syns2 = wn.synsets(word2, pos=pos_tag)
        
        for sense1, sense2 in product(syns1, syns2):
            if sense1.pos() == 's':
                sense1 = sense1.similar_tos()[0]
            if sense2.pos() == 's':
                sense2 = sense2.similar_tos()[0]

            score = wn.path_similarity(sense1, sense2)
            if score >0.65:
                true_set.add(word1+" "+word2)
                return True
    false_set.add(word1+" "+word2)
    return False

def find_synonyms(word, keyword_set, word_syn_set):
    pos_tags = [wn.NOUN, wn.ADJ]
    word_syns = set()
    for pos_tag in pos_tags:
        for ss in wn.synsets(word, pos=pos_tag):
            common = set(ss.lemma_names()).intersection(keyword_set)
            word_syns.update(common)

    filtered = [x for x in keyword_set if is_synonym(word, x, word_syn_set)]
    word_syns.update(filtered)
    return word_syns


In [10]:
game_keyphrases = set()
with open('keyphrases_dict.json', 'r', encoding='utf8') as in_json_file:
    file = json.load(in_json_file)
    game_keyphrases = set(file.keys())
#     game_keyphrases = list(game_keyphrases)[:100]
#     game_keyphrases = set(game_keyphrases)
    
movie_keyphrases = set()
with open('movie_keyphrases_dict.json', 'r', encoding='utf8') as in_json_file:
    file = json.load(in_json_file)
    movie_keyphrases = set(file.keys())
    
total_keyphrases = set()
total_keyphrases.update(game_keyphrases)
total_keyphrases.update(movie_keyphrases)

In [11]:
keyphrase_split = {keyphrase: set(keyphrase.split()) for keyphrase in total_keyphrases}

word_to_keyphrase = dict()

for (keyphrase, split) in keyphrase_split.items():
    for word in split:
        (word, pos_tag) = nltk.pos_tag([word])[0]
        if pos_tag[:2] == 'NN':
            word = WordNetLemmatizer().lemmatize(word, pos='n')
        s = word_to_keyphrase.get(word, set())
        s.add(keyphrase)
        word_to_keyphrase[word] = s

word_to_keyphrase

{'great': {'great action',
  'great actor',
  'great actors',
  'great art',
  'great atmosphere',
  'great campaign',
  'great cast',
  'great characters',
  'great chemistry',
  'great combat',
  'great community',
  'great concept',
  'great deal',
  'great depression',
  'great director',
  'great experience',
  'great fighting',
  'great film',
  'great films',
  'great friends',
  'great fun',
  'great gameplay',
  'great graphics',
  'great horror',
  'great idea',
  'great job',
  'great man',
  'great mod',
  'great mods',
  'great movie',
  'great multiplayer',
  'great music',
  'great performance',
  'great performances',
  'great physics',
  'great play',
  'great potential',
  'great puzzle',
  'great puzzles',
  'great racing',
  'great remake',
  'great remaster',
  'great rpg',
  'great rts',
  'great sequel',
  'great sim',
  'great software',
  'great soundtrack',
  'great stealth',
  'great story',
  'great survival',
  'great time',
  'great tower',
  'great voice'

In [12]:
words = word_to_keyphrase.keys()

word_to_syns = dict()
print(len(words))
counter = 0
for word in words:
    syns = word_to_syns.get(word, set())

    res = find_synonyms(word, words, syns)
    
    syns.update(res)
    word_to_syns[word] = syns
    
    for syn in res:
        syn_syns = word_to_syns.get(syn, set())
        syn_syns.add(word)
        word_to_syns[syn] = syn_syns    
    
    if counter%10==0:
        print(counter)
    counter+=1
    
word_to_syns


3625
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2

{'great': {'best',
  'beta',
  'better',
  'big',
  'bigger',
  'biggest',
  'broad',
  'burning',
  'capital',
  'central',
  'deep',
  'double',
  'epic',
  'essential',
  'fantastic',
  'giant',
  'good',
  'grand',
  'great',
  'greatest',
  'heavy',
  'hot',
  'huge',
  'important',
  'jumbo',
  'keener',
  'key',
  'large',
  'larger',
  'main',
  'master',
  'one',
  'outstanding',
  'rare',
  'remarkable',
  'solid',
  'special',
  'super',
  'superb',
  'terrific',
  'uncanny',
  'wonderful'},
 'superb': {'ace',
  'best',
  'better',
  'boss',
  'brilliant',
  'capital',
  'choice',
  'excellent',
  'good',
  'great',
  'greatest',
  'hot',
  'keener',
  'outstanding',
  'premium',
  'prime',
  'prize',
  'quality',
  'solid',
  'super',
  'superb',
  'supreme'},
 'remarkable': {'fantastic',
  'grand',
  'great',
  'greatest',
  'important',
  'large',
  'larger',
  'one',
  'rare',
  'remarkable',
  'significant',
  'special',
  'terrific',
  'uncanny',
  'wonderful'},
 'huge

In [13]:
jsondump = {word:list(syns) for (word,syns) in word_to_syns.items()}
output_path = 'phrase_word_to_syns.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(jsondump, indent=4)+'\n')

In [14]:
word_to_synphrase = dict()

for word in word_to_syns.keys():
    if word in word_to_synphrase:
        continue
    
    syns = word_to_syns[word]
    
    agg_synphrase = word_to_keyphrase[word]
    for syn in syns:
        agg_synphrase.update(word_to_keyphrase[syn])
    
    word_to_synphrase[word] = agg_synphrase
    for syn in syns:
        word_to_synphrase[syn] = agg_synphrase

word_to_synphrase
# word_to_synphrase = dict()
# for word in words:
#     # Default synphrase set is keyphrase set for this word
#     synphrase = word_to_synphrase.get(word, word_to_keyphrase[word])
    
#     word_syns = find_synonyms(word, words)
    
#     # Aggregate synphrase
#     for syn in word_syns:
#         synphrase.update(word_to_keyphrase[syn])
    
#     # Update curr word
#     word_to_synphrase[word] = synphrase
    
#     # Update all syn
#     for syn in word_syns:
#         word_to_synphrase[syn] = synphrase

# word_to_synphrase

{'great': {'best action',
  'best actor',
  'best actress',
  'best assassin',
  'best battle',
  'best bond',
  'best borderlands',
  'best british',
  'best card',
  'best city',
  'best cod',
  'best combat',
  'best comedies',
  'best comedy',
  'best comic',
  'best disney',
  'best documentaries',
  'best documentary',
  'best dragon',
  'best duty',
  'best efforts',
  'best experience',
  'best fallout',
  'best farming',
  'best fighting',
  'best film',
  'best films',
  'best flight',
  'best fnaf',
  'best fps',
  'best friend',
  'best friends',
  'best horror',
  'best idle',
  'best indie',
  'best marvel',
  'best mmo',
  'best moments',
  'best movie',
  'best movies',
  'best performance',
  'best performances',
  'best person',
  'best picture',
  'best pixar',
  'best platformer',
  'best platformers',
  'best played',
  'best puzzle',
  'best racing',
  'best rhythm',
  'best rogue',
  'best roguelike',
  'best role',
  'best rpg',
  'best rpgs',
  'best rts',
  'b

In [15]:
jsondump = {word:list(synphrase) for (word,synphrase) in word_to_synphrase.items()}
output_path = 'phrase_word_to_synphrase.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(jsondump, indent=4)+'\n')